# MAST30034 Project 2

## Data Pre-processing

In [26]:
import pandas as pd
import requests
import numpy as np
import re
import os
import json
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from geopy import distance

### External Data

#### 1. SA2 Shapefile

In [27]:
sf = gpd.read_file("../data/raw/SA2/SA2_2021_AUST_GDA2020.shp")
# only leave useful cols
SA2_inf = ['SA2_CODE21', 'SA2_NAME21', 'geometry']
sf = sf[SA2_inf]
sf.crs = 'EPSG: 4326'
#sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

#### 2. Population

In [28]:
df_popu = pd.read_excel('../data/raw/population.xlsx', sheet_name='Table 1')
# Delete the rows that has Nan in Unnamed 1 column
df_popu = df_popu.dropna(subset=["Unnamed: 1"]) 

# Change the name of columns to the first row of the dataframe
import numpy as np
# Get the first row data
array = np.array(df_popu)
list = array.tolist()
list = list[0]

# Change the column name to the first row data
df_popu.columns = list
# Delete the first row 
df_popu.drop([6], inplace=True)

# Reset index
df_popu.reset_index(drop=True, inplace=True)


df_popu = df_popu.drop(['S/T code', 'S/T name', 'GCCSA code', 'GCCSA name', 'SA4 code', 
                  'SA4 name', 'SA3 code', 'SA3 name'], axis=1)
# Change the name of each columns 
df_popu.columns = ['SA2 code','SA2 name', '2001', '2002', '2003', '2004' , 
                  '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', 
                  '2014', '2015', '2016','2017', '2018', '2019', '2020', '2021','NaN','change 01-21', 
                  'change %', 'NaN', 'km2', 'persons/km2']

# Only keep the useful columns 
df_popu = df_popu[['SA2 code', 'SA2 name', '2016', '2017', '2018', '2019', '2020', '2021']]
df_popu['SA2_code'] = df_popu['SA2 code']

In [29]:
popu_rate = []
for i in df_popu.index:
    if df_popu['2018'][i] != 0:
        popu_rate.append(float((df_popu['2021'][i]-df_popu['2018'][i])/df_popu['2018'][i]))
    else:
        popu_rate.append('NaN')     
df_popu['popu_rate18-21'] = np.array(popu_rate).tolist()
#df_popu['popu_rate18-21'] = pd.to_numeric(popu_rate['popu_rate18-21'])

popu2022 = []
popu2025 = []
for i in df_popu.index:
    if df_popu['popu_rate18-21'][i] == 'NaN':
        popu2022.append(df_popu['2019'][i])
    else:
        popu2022.append(df_popu['2019'][i] + df_popu['2019'][i]*float(df_popu['popu_rate18-21'][i]))
df_popu['popu2022'] = np.array(popu2022).tolist()
for i in df_popu.index:
    if df_popu['popu_rate18-21'][i] == 'NaN':
        popu2025.append(df_popu['popu2022'][i])
    else:
        popu2025.append(df_popu['popu2022'][i] + df_popu['popu2022'][i]*float(df_popu['popu_rate18-21'][i]))
df_popu['popu2025'] = np.array(popu2025).tolist()
lst = ['SA2_code', 'SA2 name', 'popu2022', 'popu2025']
df_popu = df_popu[lst]

#### 3. Income

In [30]:
df_income = pd.read_excel('../data/raw/income.xlsx', sheet_name='Table 1')

df_income = df_income.dropna(subset=["Unnamed: 1"]) 
df_income = df_income.iloc[3186:19235] # the income instance counted by SA2 code
att = ['            Australian Bureau of Statistics','Unnamed: 2', 'Unnamed: 7']
df_income = df_income[att]
df_income = df_income.reset_index(drop = True)
df_income.rename(columns = {'            Australian Bureau of Statistics':'SA2_code', 'Unnamed: 2':'Year', 
                            'Unnamed: 7':'Mean Income'}, inplace = True)

df_income_2014 = df_income.loc[df_income['Year'] == 2014]
df_income_2016 = df_income.loc[df_income['Year'] == 2016]
df_income_2017 = df_income.loc[df_income['Year'] == 2017]

df_income = pd.merge(df_income_2014, df_income_2016, on='SA2_code')
df_income = pd.merge(df_income, df_income_2017, on='SA2_code')

df_income.rename(columns = {'Mean Income_x':'Mean Income2014', 'Mean Income_y': 'Mean Income2016', 
                            'Mean Income':'Mean Income2017'}, inplace = True)

#### 4. PTV

In [31]:
stops_2 = pd.read_csv("../data/raw/ptv/2/stops.txt")
stops_2 = stops_2.loc[:,["stop_name","stop_lat","stop_lon"]]
stops_3 = pd.read_csv("../data/raw/ptv/3/stops.txt")
stops_3 = stops_3.loc[:,["stop_name","stop_lat","stop_lon"]]
stops_4 = pd.read_csv("../data/raw/ptv/4/stops.txt")
stops_4 = stops_4.loc[:,["stop_name","stop_lat","stop_lon"]]

gdf_stops_2 = gpd.GeoDataFrame(
    stops_2, geometry=gpd.points_from_xy(stops_2['stop_lon'], stops_2['stop_lat']))
gdf_stops_3 = gpd.GeoDataFrame(
    stops_3, geometry=gpd.points_from_xy(stops_3['stop_lon'], stops_3['stop_lat']))
gdf_stops_4 = gpd.GeoDataFrame(
    stops_4, geometry=gpd.points_from_xy(stops_4['stop_lon'], stops_4['stop_lat']))

gdf_stops_2.crs = 'EPSG: 4326'
gdf_stops_3.crs = 'EPSG: 4326'
gdf_stops_4.crs = 'EPSG: 4326'
stops2_sa2 = gdf_stops_2.sjoin(sf, how='inner', predicate='within')
stops3_sa2 = gdf_stops_3.sjoin(sf, how='inner', predicate='within')
stops4_sa2 = gdf_stops_4.sjoin(sf, how='inner', predicate='within')

geoJSON_2 = stops2_sa2[['index_right', 'geometry']].drop_duplicates('index_right').to_json()
geoJSON_3 = stops3_sa2[['index_right', 'geometry']].drop_duplicates('index_right').to_json()
geoJSON_4 = stops4_sa2[['index_right', 'geometry']].drop_duplicates('index_right').to_json()

stops2_df = stops2_sa2.groupby('SA2_CODE21')['geometry'].count()
stops2_df = stops2_df.to_frame().reset_index()
stops2_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'stop_count'}, inplace = True)

stops3_df = stops3_sa2.groupby('SA2_CODE21')['geometry'].count()
stops3_df = stops3_df.to_frame().reset_index()
stops3_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'stop_count'}, inplace = True)

stops4_df = stops4_sa2.groupby('SA2_CODE21')['geometry'].count()
stops4_df = stops4_df.to_frame().reset_index()
stops4_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'stop_count'}, inplace = True)

stops2_sa2 = pd.merge(left=stops2_sa2, right=stops2_df, left_on='SA2_CODE21', right_on='SA2_code')

att = ['SA2_code', 'stop_count']
stops2_sa2_count = stops2_sa2[att]

#### 5. School

In [32]:
df_school = pd.read_csv('../data/raw/school2021.csv')

# Filter the columns and rename them.
df_school = df_school[['X', 'Y', 'Address_Line_1', 'School_Name']]  # type: ignore
df_school.rename(columns={'X':'school_X', 'Y':'school_Y',
                          'Address_Line_1':'school_addr', 'School_Name': 'school_name'}, inplace = True)

# now change our school dataframe to geodataframe by transforming the x, y coordinates to point geometry.
gdf_school = gpd.GeoDataFrame(
    df_school, geometry=gpd.points_from_xy(df_school['school_X'], df_school['school_Y']))

gdf_school.crs = 'EPSG: 4326'

# Then apply spatial join
school_sa2 = gdf_school.sjoin(sf, how='inner', predicate='within')

school_df = school_sa2.groupby('SA2_CODE21')['geometry'].count()
school_df = school_df.to_frame().reset_index()
school_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'school_count'}, inplace = True)

#### 6. Hospital

In [33]:
df_hospital = pd.read_csv('../data/raw/hospital2021.csv')
# Filter the columns and rename them.
df_hospital = df_hospital[['X','Y','addr_street','name']]
df_hospital.rename(columns={'X':'hospital_X', 'Y':'hospital_Y', 'addr_street':'hospital_addr','name':'hospital_name'}, inplace = True)

# Remove the null value.
df_hospital = df_hospital.dropna(axis=0, subset=['hospital_X', 'hospital_Y'])
# now change our hospital dataframe to geodataframe by transforming the x, y coordinates to point geometry.
gdf_hospital = gpd.GeoDataFrame(
    df_hospital, geometry=gpd.points_from_xy(df_hospital['hospital_X'], df_hospital['hospital_Y']))

gdf_hospital.crs = 'EPSG: 4326'

# Then apply spatial join
hospital_sa2 = gdf_hospital.sjoin(sf, how='inner', predicate='within')

hospital_df = hospital_sa2.groupby('SA2_CODE21')['geometry'].count()
hospital_df = hospital_df.to_frame().reset_index()
hospital_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'hos_count'}, inplace = True)

### Domain Rental Prices

In [34]:
df_domain = pd.read_json('../data/raw/domain1.json')

df_domain = df_domain.T
df_domain = df_domain.drop('desc', axis = 1)
df_domain = df_domain[df_domain['rooms'].astype(bool)]
df_domain = df_domain.reset_index(drop = True)

bed = []
bath = []
parking = []
Longitude = []
Latitude = []
postcode = []

for i in df_domain.index:
    
    postcode.append(df_domain['name'][i][-4:])
    
    for r in ((',', ''), ('$', ''), ('.00', '')):
        df_domain['cost_text'][i] = df_domain['cost_text'][i].replace(*r)
    # convert room col: [bed, bath, parking] to individual col
    bed.append(df_domain['rooms'][i][0][0])
    bath.append(df_domain['rooms'][i][1][0]) 
    if len(df_domain['rooms'][i]) == 3:    # if no parking info, default no parkings      
        parking.append(df_domain['rooms'][i][2][0])
    else:     
        parking.append('0')
    Longitude.append(df_domain['coordinates'][i][0])
    Latitude.append(df_domain['coordinates'][i][1])

df_domain['Postcode'] = np.array(postcode).tolist()
df_domain['Prices'] = df_domain['cost_text'].str.extract('(\d+)') 
df_domain['Prices'] = pd.to_numeric(df_domain['Prices'])
df_domain['Bedrooms'] = np.array(bed).tolist()
df_domain['Bathrooms'] = np.array(bath).tolist()
df_domain['Parkings'] = np.array(parking).tolist()
df_domain['Longitude'] = Longitude
df_domain['Latitude'] = Latitude

df_domain = df_domain.drop('rooms', axis = 1)
df_domain = df_domain.drop('cost_text', axis = 1)
df_domain = df_domain.dropna().reset_index(drop = True)
df_domain = df_domain.drop('coordinates', axis = 1)

# remove extreme prices
df_domain = df_domain[(df_domain['Prices'] > 100) & (df_domain['Prices'] < 10000)]
# remove duplicate
df_domain = df_domain.drop_duplicates()
df_domain = df_domain.reset_index(drop = True)

### Data Aggregation

In [35]:
# build geodataframe
gpd_domain = gpd.GeoDataFrame(
    df_domain, geometry=gpd.points_from_xy(df_domain.Latitude, df_domain.Longitude))

# standardlize crs
gpd_domain.crs = "EPSG:4326"

gpd_domain = gpd_domain.sjoin(sf, how="inner", predicate='within')
gpd_domain = gpd_domain.reset_index(drop = True)

gpd_domain['SA2_code'] = gpd_domain['SA2_CODE21']
gpd_domain = gpd_domain.drop('SA2_CODE21', axis = 1)

# merge population by SA2
df_popu['SA2_code'] = df_popu['SA2_code'].apply(str)
gpd_domain = pd.merge(gpd_domain, df_popu, on='SA2_code')

# merge income by SA2
df_income['SA2_code'] = df_income['SA2_code'].apply(str)
gpd_domain = pd.merge(gpd_domain, df_income, on='SA2_code')
inco_rate = []
for i in gpd_domain.index:
    inco_rate.append(float((gpd_domain['Mean Income2017'][i] - gpd_domain['Mean Income2014'][i])/
                           gpd_domain['Mean Income2014'][i]))
gpd_domain['Income rate14-17'] = np.array(inco_rate).tolist()
inco2019 = []
inco2022 = []
inco2025 = []
for i in gpd_domain.index:
    inco2019.append(gpd_domain['Mean Income2016'][i] + gpd_domain['Mean Income2016'][i]*
                    float(gpd_domain['Income rate14-17'][i]))
gpd_domain['Mean Income2019'] = np.array(inco2019).tolist()
for i in gpd_domain.index:
    inco2022.append(gpd_domain['Mean Income2019'][i] + gpd_domain['Mean Income2019'][i]*
                    float(gpd_domain['Income rate14-17'][i]))
gpd_domain['Mean Income2022'] = np.array(inco2022).tolist()
for i in gpd_domain.index:
    inco2025.append(gpd_domain['Mean Income2022'][i] + gpd_domain['Mean Income2022'][i]*
                    float(gpd_domain['Income rate14-17'][i]))
gpd_domain['Mean Income2025'] = np.array(inco2025).tolist()

In [36]:
#gpd_domain = pd.merge(gpd_domain, stops2_sa2_count, on='SA2_code')
#gpd_domain = pd.merge(gpd_domain, hospital_df, on='SA2_code')
#gpd_domain = pd.merge(gpd_domain, school_df, on='SA2_code')

In [37]:
att = ['name', 'Postcode', 'Prices', 'Bedrooms', 'Bathrooms', 'Parkings', 'geometry', 'SA2_code', 
       'Longitude', 'Latitude', 'popu2022', 'popu2025', 'Mean Income2022', 
      'Mean Income2025'] #'school_count', 'stop_count', 'hos_count'
gpd_domain = gpd_domain[att]

In [38]:
gpd_domain

,name,Postcode,Prices,Bedrooms,Bathrooms,Parkings,geometry,SA2_code,Longitude,Latitude,popu2022,popu2025,Mean Income2022,Mean Income2025
0,1/62 Simpson St East Melbourne VIC 3002,3002,395.0,1,1,0,POINT (144.98911 -37.81067),206041119,-37.810666,144.989108,4965.669440,4555.306297,98664.943495,102957.654101
1,29/8 Wellington Crescent East Melbourne VIC 3002,3002,750.0,2,2,2,POINT (144.97767 -37.81722),206041119,-37.817224,144.977670,4965.669440,4555.306297,98664.943495,102957.654101
2,811v/162 Albert Street East Melbourne VIC 3002,3002,600.0,2,1,1,POINT (144.98553 -37.81004),206041119,-37.810043,144.985531,4965.669440,4555.306297,98664.943495,102957.654101
3,203/33 Cliveden Close East Melbourne VIC 3002,3002,380.0,1,1,1,POINT (144.97672 -37.81710),206041119,-37.817100,144.976716,4965.669440,4555.306297,98664.943495,102957.654101
4,9/322 Albert Street East Melbourne VIC 3002,3002,990.0,3,2,1,POINT (144.98027 -37.80984),206041119,-37.809840,144.980274,4965.669440,4555.306297,98664.943495,102957.654101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4031,43 Bellman Avenue Clyde VIC 3978,3978,450.0,3,2,2,POINT (145.34012 -38.12376),212031303,-38.123765,145.340118,21129.675792,34771.277186,71382.469390,79738.921375
4032,11 Bonfire Way Cranbourne South VIC 3977,3977,450.0,3,2,1,POINT (145.23057 -38.13111),212031304,-38.131111,145.230571,23184.689777,27207.057756,59706.852893,65347.627829
4033,2 Creekview Way Cranbourne West VIC 3977,3977,500.0,4,2,2,POINT (145.25386 -38.10221),212031304,-38.102212,145.253859,23184.689777,27207.057756,59706.852893,65347.627829
4034,214 Evans Road Cranbourne West VIC 3977,3977,400.0,3,2,2,POINT (145.25655 -38.09045),212031304,-38.090450,145.256545,23184.689777,27207.057756,59706.852893,65347.627829


## By Suburb 

(below are quoted from MAST30034 subject group)

"While ABS offers data such as population in SA2 level, metrics/rankings needs to be produced with regard to Suburbs (postcodes).

In this research, we assume that: "ratio between properties counts in neighbouring areas is a good estimator of the ratio between their populations". Below is an example applying this assumption:

Given two neighbouring suburbs X, Y in the same SA2 region Z with 100,000 population. If there are 600 listed properties in X, and 400 in Y, then the population in X and Y can be reasonably estimated as 60,000 and 40,000, respectively."

In [39]:
# Group by SA2_code and then Postcode and then count, to see the property ratio of a suburb in its SA2 district.

df = gpd_domain.groupby(['SA2_code', 'Postcode']).count()
df

name  Prices  Bedrooms  Bathrooms  Parkings  geometry  \
SA2_code  Postcode                                                          
109031185 3644         2       2         2          2         2         2   
201011001 3350         4       4         4          4         4         4   
201011002 3350         4       4         4          4         4         4   
          3355         3       3         3          3         3         3   
201011005 3357         2       2         2          2         2         2   
...                  ...     ...       ...        ...       ...       ...   
217041478 3289         1       1         1          1         1         1   
217041479 3280         9       9         9          9         9         9   
217041480 3277         1       1         1          1         1         1   
          3280        11      11        11         11        11        11   
401021009 3807         1       1         1          1         1         1   

                    Longitude  Latitude  popu2022  popu2025  Mean Income2022  \
SA2_code  Postcode                                                             
109031185 3644              2         2         2         2                2   
201011001 3350              4         4         4         4                4   
201011002 3350              4         4         4         4                4   
          3355              3         3         3         3                3   
201011005 3357              2         2         2         2                2   
...                       ...       ...       ...       ...              ...   
217041478 3289              1         1         1         1                1   
217041479 3280              9         9         9         9                9   
217041480 3277              1         1         1         1                1   
          3280             11        11        11        11               11   
401021009 3807              1         1         1         1                1   

                    Mean Income2025  
SA2_code  Postcode                   
109031185 3644                    2  
201011001 3350                    4  
201011002 3350                    4  
          3355                    3  
201011005 3357                    2  
...                             ...  
217041478 3289                    1  
217041479 3280                    9  
217041480 3277                    1  
          3280                   11  
401021009 3807                    1  

[550 rows x 12 columns]

In [40]:
# Calculate the ratio.

pct = lambda x: x / x.sum()

out = df.groupby(['SA2_code', 'Postcode']).sum().groupby('SA2_code').apply(pct)
pct_ = out['name']

pct_

SA2_code   Postcode
109031185  3644        1.000000
201011001  3350        1.000000
201011002  3350        0.571429
           3355        0.428571
201011005  3357        1.000000
                         ...   
217041478  3289        0.250000
217041479  3280        1.000000
217041480  3277        0.083333
           3280        0.916667
401021009  3807        1.000000
Name: name, Length: 550, dtype: float64

In [41]:
# create an array for property ratio.

array = np.empty(len(gpd_domain))

#iterate through rows of gpd_domain and assign each line with its ratio.

for i, row in gpd_domain.iterrows():
    array[i] = pct_.loc[row['SA2_code'], row['Postcode']]
       
ser = pd.Series(array)

# add the new column to dataframe.
gpd_domain['pct'] = ser

In [42]:
gpd_domain.head(30)

,name,Postcode,Prices,Bedrooms,Bathrooms,Parkings,geometry,SA2_code,Longitude,Latitude,popu2022,popu2025,Mean Income2022,Mean Income2025,pct
0,1/62 Simpson St East Melbourne VIC 3002,3002,395.0,1,1,0,POINT (144.98911 -37.81067),206041119,-37.810666,144.989108,4965.669440,4555.306297,98664.943495,102957.654101,1.000000
1,29/8 Wellington Crescent East Melbourne VIC 3002,3002,750.0,2,2,2,POINT (144.97767 -37.81722),206041119,-37.817224,144.977670,4965.669440,4555.306297,98664.943495,102957.654101,1.000000
2,811v/162 Albert Street East Melbourne VIC 3002,3002,600.0,2,1,1,POINT (144.98553 -37.81004),206041119,-37.810043,144.985531,4965.669440,4555.306297,98664.943495,102957.654101,1.000000
3,203/33 Cliveden Close East Melbourne VIC 3002,3002,380.0,1,1,1,POINT (144.97672 -37.81710),206041119,-37.817100,144.976716,4965.669440,4555.306297,98664.943495,102957.654101,1.000000
4,9/322 Albert Street East Melbourne VIC 3002,3002,990.0,3,2,1,POINT (144.98027 -37.80984),206041119,-37.809840,144.980274,4965.669440,4555.306297,98664.943495,102957.654101,1.000000
5,107K/211 Powlett Street East Melbourne VIC 3002,3002,550.0,1,1,1,POINT (144.98599 -37.80970),206041119,-37.809699,144.985994,4965.669440,4555.306297,98664.943495,102957.654101,1.000000
6,4/23 Albert Street East Melbourne VIC 3002,3002,475.0,1,1,0,POINT (144.97739 -37.80978),206041119,-37.809776,144.977391,4965.669440,4555.306297,98664.943495,102957.654101,1.000000
7,7/126 Wellington Parade East Melbourne VIC 3002,3002,1600.0,3,2,1,POINT (144.98592 -37.81621),206041119,-37.816211,144.985925,4965.669440,4555.306297,98664.943495,102957.654101,1.000000
8,1105/162 Albert Street East Melbourne VIC 3002,3002,450.0,2,1,1,POINT (144.98553 -37.81004),206041119,-37.810043,144.985531,4965.669440,4555.306297,98664.943495,102957.654101,1.000000
9,23/2 Wellington Crescent East Melbourne VIC 3002,3002,750.0,2,2,2,POINT (144.97754 -37.81760),206041119,-37.817596,144.977542,4965.669440,4555.306297,98664.943495,102957.654101,1.000000


## Distance

In [44]:
import openrouteservice as ors
import folium
import time

In [45]:
ors_key = "5b3ce3597851110001cf624852ff1ab2e1224e1d9d57e2ad0f541f1a"
client = ors.Client(key=ors_key)

In [46]:
distance = [2.640392620451656,2.127094055053294,2.317093524029894,1.993894427328646,1.863594791498904,2.11449409026853,1.49949580910743,2.796792183335552,2.317093524029894,2.17249392816665,2.2368937481776663,2.050894268021626,2.278393632190976,3.287590811618264,1.72399518166136,2.201293847674682,2.2853936126269563,2.9067918759009523,2.186693888479638,1.72399518166136,4.195388274444356,4.797186592497607,4.44698757125758,4.211088230565054,3.604589925647644,5.425284837046042,5.905583494674785,5.537184524301208,5.341285071814282,5.365185005017128,5.059685858846858,5.898883513400346,5.800283788973542,5.99548324341687,5.597084356889094,5.147085614576095,5.8459836612484395,5.514184588582988,5.811283758230083,5.199185468963888,6.093582969241104,5.945883382041926,6.19548268444487,5.392684928158478,5.24148535074131,6.170782753477912,5.042685906359478,5.166685559796838,5.042685906359478,5.122085684447595,3.9199890441488003,4.447087570978095,4.320087925925314,4.801186581318168,4.416787655662352,4.364687801274558,4.211588229167624,3.00799159306112,2.811892141133166,3.0246915463869577,3.732189569023508,3.2333909630996764,3.626289864999182,3.537190114021208,4.397587709323664,3.626289864999182,3.242790936827992,2.811892141133166,2.977791677465892,2.7137924153089323,3.3587906126242326,2.720092397701314,2.815692130512698,2.8421920564489076,2.5302929281657423,5.84698365845358,7.46897912519066,6.433282019827162,6.424782043583472,6.433282019827162,6.433282019827162,6.424782043583472,6.433282019827162,6.433282019827162,6.433282019827162,6.424782043583472,6.433282019827162,6.433282019827162,8.353276653695962,8.353276653695962,8.353276653695962,6.421282053365482,7.09898015928886,7.248079742575234,8.944675000815758,8.290476829213171,9.38247377722605,9.455473573201271,10.040671937649199,9.92847225123249,9.92847225123249,8.61747591529395,9.054074695058075,8.726375610933696,9.399873728595486,9.508673424514718,11.077169040776809,10.95796937392412,9.673772963083332,10.00197204581028,9.38577376800301,11.456867979568464,9.288074041060835,12.35396547229956,12.42596527106964,12.25446575038813,12.818564173807605,12.364865441835585,11.451767993822251,12.455365188900755,12.479965120147199,12.594064801253674,11.484567902150845,12.47876512350103,12.359765456089372,12.27146570287551,12.223065838146734,12.07046626464237,12.17196598096408,11.692667320540478,12.416665297061838,12.450165203434029,11.812766984877792,12.7349644074579,14.275160102814528,14.638859086323945,13.077663449659381,15.28025729370074,15.300557236965085,13.181363159832397,14.097360599740636,13.718761657874632,14.661459023160111,14.193860330036646,13.703661700077019,13.31976277302377,13.589062020367974,14.774058708458876,13.949861011982485,13.92226108912062,13.585762029591013,13.9099611234974,14.15016045217203,15.014058037692475,8.03797753491532,11.659267413888802,8.327276726362323,10.380870986837825,8.381576574601425,11.106768958048951,10.265971307967241,7.813578162081905,10.020771993266912,10.721270035467482,11.773267095274761,10.763669916965418,9.014574805455045,7.506579020103924,11.641667463078338,8.814775363868073,10.022571988236164,10.182371541617536,8.160777191706512,8.468876330610145,11.85246687392185,11.474267930937902,12.31546557990167,13.54796213523672,13.35166268386774,11.206068680519355,13.28196287866948,10.08147182361891,12.95046380516557,10.613470336753391,13.30146282416971,13.12346332165479,12.141566065927824,10.457670772192579,11.172368774706136,10.448370798184776,12.939963834511602,10.820669757658399,12.885463986831471,12.18896593345146,13.527062193649293,14.574259266871902,14.862758460554792,13.75646155250841,14.15696043316698,14.32195997201508,12.898063951616235,13.75026156983654,12.34316550248405,13.527062193649293,13.106063370285353,11.584067624062275,13.539062160110976,13.852561283922364,13.60026198906554,13.813861392083446,13.635361890965955,12.25096576017014,12.949363808239918,12.284565666262845,12.465165161511129,18.561348123485597,17.78805028475083,20.838741758471432,20.75104200358065,20.298043269652233,17.501051086875652,19.288746090504432,18.485348335894958,18.72524766540804,18.77124753684448,21.0792410863076,18.98644693539061,20.72114208714697,20.45964281800286,17.89914997424189,20.823341801512278,18.62094795691194,18.136849309903667,20.943241466408562,20.655842269651327,13.55246212265985,12.902963937921418,14.242860193088505,13.403362539373475,14.278460093591491,25.41092897981254,23.832533391219563,23.141035323865253,23.298034885072234,23.23273506757659,23.180735212909312,23.93963309189006,23.63243395047105,22.785136318555928,23.65813387864315,23.87543327132007,23.155135284457728,21.84513894572433,22.854336125151615,19.24314621795005,21.518639858246118,21.240840634658227,20.42774290715889,21.455540034601782,20.843541745056104,13.520362212374856,13.791761453849853,13.791761453849853,13.854861277494187,14.146560462233523,22.33033758965826,22.34723754242512,21.33074038340031,23.156835279706467,22.088238266293864,23.11833538730858,20.778441927001488,22.242537835046964,22.40513738060273,23.156435280824407,21.82543900078307,22.91223596332922,22.05203836746779,21.462540015037764,23.42913451866609,33.535006274090414,25.16172967629165,25.64212833364091,22.0685383213526,22.08713826936821,24.15733248344904,24.840730573441714,25.378829069527548,23.106635420008438,25.07382992195985,24.62313118160325,25.382829058348108,25.61222841720722,25.20072956729211,25.61222841720722,31.698411407130287,31.617611632954976,31.556011805118356,31.26381262177645,29.26901819696317,31.670211485945345,31.589311712049522,29.35881794598475,32.144310160902215,28.6185200150196,32.72910852646809,33.13540739091647,32.70090860528314,45.256973512741496,44.102276739966335,44.785674829959014,38.44159256083034,44.025576954332095,42.059882448188404,42.57258101526368,37.113096273801844,42.40818147473866,25.32712921402181,24.556331368299897,25.356329132411897,25.050829986241627,24.79843069166429,24.58583128585153,24.40433179311862,24.875930475062642,24.51343148819939,24.556331368299897,24.645031120395814,24.90473039457067,24.58583128585153,24.80253068020536,25.292729310164994,21.09544104103087,20.354243112581102,19.711344909396598,21.584539674064843,20.78964189569906,21.80103906897765,20.798641870545318,24.133732549407732,23.79983348261149,23.50713430066701,24.20383235348805,24.06863273135312,24.849730548287972,23.89973320340497,24.598831249518348,23.73693365840818,24.081532695299423,24.58303129367713,25.153029700606933,26.167826864383006,35.44760092864118,37.676494699177724,37.32359568548382,45.204073660589586,37.33209566172751,45.46707292554141,46.76286930396182,39.848288629300775,46.40337030871399,40.12078786770143,46.74136936405131,37.53589509213504,37.52659511812723,41.771283254785,37.37689553651778,46.86186902727068,45.11657390513984,45.29187341520088,32.27520979505504,31.750311262077062,31.750311262077062,31.750311262077062,38.60869209380923,39.047590867145175,37.067196402085926,38.469492482853745,38.33279286491111,38.36039278777297,31.221412740278506,31.831611034854937,32.330009641896716,30.601914471694283,29.397717837264693,31.13201299013899,32.06151039231662,32.955807892873324,33.25790704854612,32.59410890377419,32.48000922266771,32.390009474205115,33.94430513015422,30.116815827480867,34.83620263741859,38.55639223998041,30.88181368941297,33.20020720980955,29.160118501323428,4.88548634571147,4.6710869449294545,6.0225831676761645,4.96398612631496,5.113185709321848,4.22948817913963,5.274185259349388,5.024685956666958,5.538284521226862,5.080085801831714,6.08198300166148,4.844186461139188,4.354687829223158,5.142285627991422,5.701584064826224,4.884786347667872,4.812686549177277,6.055683075166298,5.14198562882988,4.466487516757811,5.584584391824845,6.041283115412282,9.81747256146195,9.81747256146195,9.532973356599621,8.494876257943785,8.971374926192995,9.532173358835509,9.93897222188646,9.96347215341239,6.148782814964832,5.409784880366372,7.617978708756521,7.721678418929539,8.37197660143208,7.503879027650045,6.844780869742272,4.921586244817025,7.5796788157996575,5.555684472596298,7.196579886510524,20.699042148913374,20.95094144488814,19.13364652398722,18.56174812236765,18.91164714444614,19.025046827509012,17.60725079006152,17.28955167798854,18.035249593861444,26.71432533699202,26.00522731882724,26.973124613682245,26.66292548064782,25.95822745018566,26.880124873604228,29.49281757147351,26.98712457455421,27.72112252312697,27.038224431736865,27.009224512787803,27.55092299881214,27.638722753423433,24.01603287836275,23.781833532918967,24.18443240770833,24.730830880596827,6.763381097243876,7.046280306577982,8.342676683321478,8.69747569170515,6.37798217438292,7.177379940171836,7.997377648386636,7.813678161802418,7.69047850612917,7.036180334806067,8.43277643150459,5.931383422567396,6.084782993835872,7.516078993552754,7.264779695901073,6.664581373376044,6.847080863314094,5.98198328114748,7.925177850175528,11.832866928701106,10.47597072104664,10.749269957211402,12.197365909974636,11.537367754582235,11.02846917688649,12.3992653456924,10.927269459726322,12.61266474926928,11.04716912462261,12.61266474926928,10.008872026525745,10.440170821102628,13.019363612599715,11.16276880153679,12.412665308241278,12.0242663937649,11.622167517578108,15.929455479277632,11.62046752232937,15.243857395433647,15.619956344286798,14.887558391242266,15.3656570550197,14.391059778890254,14.643359073747074,14.962758181068791,16.260854553061026,15.276157305159668,16.092355023994934,15.395556971453383,15.070157880900828,19.799244663728402,20.712142112300707,16.15345485322899,19.23314624589865,16.47965394154566,16.47965394154566,16.47965394154566,20.626742350981754,20.483342751764678,20.208543519792205,20.02364403656182,19.46864558770912,19.1399465063796,20.35914309888629,19.91964432722726,20.32374319782433,19.92344431660679,19.68364498681422,20.858941702015258,20.923741520908333,17.66585062628273,17.957949809904118,19.65834505752418,19.733244848189162,19.71644489514281,19.4442456559037,19.733244848189162,21.72543928026907,21.75523919698224,20.893841604474648,17.654050659262072,17.88615001057507,16.39725417184212,16.72025326910234,9.542773329209991,9.836072509477555,8.487176279464208,9.542773329209991,9.885072372529415,8.52827616459546,14.088360624894376,13.610461960557972,13.334962730541898,14.05416072047859,16.39695417268058,15.468056768826036,16.26195454998668,15.42045690186137,16.258554559489205,15.628956319133062,15.776055908009155,15.974355353788416,15.520556622095885,15.3156571947627,15.381857009742966,17.497551096657663,16.74165320929234,15.84545571404587,17.083752253170733,17.92084991359343,19.180446393187772,19.02464682862696,18.25744897284355,19.92764430486838,19.767244753163922,14.949958216843,16.62445353684993,16.99205250945939,17.111952174355682,16.78885307737495,16.773753119577332,16.295654455799898,21.40504017574221,21.56383973191845,20.90744156646455,20.250243403246543,20.772341944050137,21.92273872884319,21.058341144720178,19.09944661957143,19.101546613702222,22.07073831520391,20.354643111463158,21.28044052398177,19.75254479424836,21.38744022493175,19.54314537949205,20.169843627953288,19.18714637446221,3.2201909999918277,3.775689447447098,2.560892842643026,3.6549897847866997,4.968886112620146,3.64548981133787,3.2201909999918277,3.938188993282348,3.497190225815608,3.772389456670136,4.532587332017565,4.058788656222232,3.64548981133787,3.24449093207673,3.3290906956315744,3.61748988959395,3.984788863041872,3.4469903661175803,3.4469903661175803,3.427790419778892,1.341796249856852,1.138296818610862,1.363396189487876,2.094594145886244,1.151896780600766,2.313793533252932,2.468393101167576,2.2298937677416864,1.138296818610862,1.138296818610862,1.430896000834826,2.120094074617314,2.2778936335884064,2.0892941606990023,2.2091938255952877,1.536895704579666,1.417696037726978,2.25799368920612,2.036094309385554,2.592692753766478,3.4930902372745343,3.009591588589344,3.213391018996876,3.5873899737192363,4.120588483499884,3.6105899088784836,3.40299048909142,3.393390515922076,3.5777900005498924,2.661392561759596,3.44799036332272,4.004588807703644,4.120188484617828,3.6019899329142797,3.297390784228636,3.181191108991368,3.1432912149165624,3.2811908295053676,3.933789005579732,3.308290753764662,5.455084753759214,8.09047738818517,7.914277880639503,6.05698307153298,8.09047738818517,8.534776146428872,7.877577983210864,8.14517723530633,8.14517723530633,8.14517723530633,5.7493839312319155,8.176877146709266,8.09047738818517,8.248376946876775,5.424484839281931,7.847278067895122,8.27077688427191,7.894377936257216,6.879480772760631,5.303085178577935,4.515687379250698,5.16898555336866,4.88098635828834,4.475887490486126,4.298187987132748,4.40298769423142,6.139282841516002,6.235082573768414,4.090388567904656,6.171282752080482,5.288385219662375,4.405787686405812,4.46948750837323,4.298187987132748,4.630787057562312,6.171282752080482,4.167788351582492,4.46948750837323,10.211671459728139,10.417370884825436,26.08392709887176,26.12972697086717,26.01912727997869,27.014924496857102,26.276326561140696,25.3806290644968,24.242532245326963,24.19723237193412,24.98723016399472,27.014924496857102,24.537831420004807,26.35292634705442,23.458834435658748,23.458834435658748,12.861364054187597,13.763861531826445,12.304365610924615,15.291057263516254,13.22446303937393,13.676761775258752,13.596861998568064,12.951663801811739,14.248560177157804,13.85576127497881,12.511765031270652,14.00146086776771,12.963463768832389,14.209660285877858,13.056363509189895,14.173060388169734,14.73945880516103,12.247365770231635,14.26246013830925,37.9126940390318,33.78730556894724,37.7126945980038,38.14809338112175,52.00855464296472,26.533325842861675,26.586425694454608,27.388523452697406,26.567525747277465,26.27812655610995,26.27812655610995,40.64148641241782,38.947291147469635,41.62648365948072,39.05169085568625,33.7035058031565,2.416193247059268,2.391393316371796,3.371690576570538,1.7017952437072519,3.3493906388959163,2.486593050301124,1.522295745384622,2.4004932909385697,2.56549282978667,2.52349294717079,2.97649168109921,1.9265946154227238,2.307393551140036,2.8593920083773163,2.284093616260274,2.704292441860102,2.5008930103346265,2.56549282978667,1.9112946581840817,2.325593500273584,2.4187932397926324,2.385693332302498,3.4400903854021143,1.976094477077154,3.1002913350955423,3.501790212959252,2.638792624923432,2.4187932397926324,4.0000888202805145,2.7723922515301362,2.783592220227704,2.985291656504442,3.453890346833046,2.3499934320789997,2.4187932397926324,3.2997907775209723,2.4187932397926324,3.65448978618413,2.595192746779328,3.5077901961900926,4.0984885452662905,4.172088339564595,4.606287126036382,4.606287126036382,4.016388774724296,4.499787423688972,4.587587178300264,3.8822891495150222,2.9235918289473037,4.606287126036382,4.0099887926113995,4.066888633583866,4.021588760191023,3.6496897995994577,4.115088498871614,4.699286866114402,3.4957902297284127,4.32698790664078,3.258690892389718,4.065688636937698,5.142585627152965,5.040485912508171,3.8635892017789044,5.040485912508171,3.9893888501855157,4.757786702615093,5.079285804067602,5.075285815247042,5.636084247889554,5.211385434866595,5.052085880087795,5.052085880087795,5.442384789253936,5.052085880087795,5.052085880087795,5.052085880087795,4.933886210440245,5.045285899092842,5.899183512561888,5.523584562311305,8.750375543857055,8.881875176332965,9.24317416655005,9.00167484150874,8.759375518703315,8.72047562742337,9.9806721053408,8.719675629659259,9.488373481250376,9.13697446336418,8.46047635408697,9.42647365425221,8.13247727080105,9.817872560344005,8.400576521499083,9.181874337874966,9.323773941284331,8.967774936254491,8.352076657049794,8.273276877284761,10.521770593042051,8.72047562742337,9.321073948830454,8.824175337596389,7.237079773318694,7.9531777719194485,8.069577446597744,9.65577301339081,8.32147674257251,8.606075947155354,8.811575372811625,7.820278143356342,8.591075989078254,7.924277852690903,9.104574553917645,9.436073627421555,8.764275505008502,8.046177511997467,7.312179563424708,9.743672767722618,9.913372293434875,8.841775288406852,9.63197307990848,9.96047216179697,8.479276301543601,9.31017397929443,10.433570839548704,10.75276994742939,11.191768720485852,10.4449708076873,9.938772222445431,9.879772387342172,10.321671152293538,8.836075304337555,11.2442685737557,10.21347145469739,10.657870212661607,9.137774461128291,9.365273825297642,9.221674226639538,12.571664863858539,18.09624942337498,16.99295250694402,16.66165343288114,17.19845193260029,18.57044809805237,16.814553005547044,16.84145293036531,16.87445283813493,17.43055128391328,17.20715190828501,17.33385155417625,17.545850961665927,17.60115080711017,16.601353601411198,15.904655548590158,17.244051805154673,16.56525370230564,16.82915296474209,17.5949508244383,17.20665190968244,23.09333545718008,22.29513768803733,22.88943602705203,23.73813365505435,20.73464204941636,21.990738538792712,21.4006401880396,21.81693902453938,20.17744360671235,21.20114074561417,20.346343134660497,21.14904089122637,20.82744179005335,22.45543724002127,22.77843633728149,27.04172442195485,24.608831221569748,22.1406381198432,10.516870606736866,12.215865858269726,13.816461384816808,11.46046796950697,11.82946693820363,13.636361888171097,11.48966788789706,11.221868636360567,11.54366773697462,13.724061643061875,12.133166089404648,13.411562516455625,11.66796738957352,10.35447106062213,11.448068004163234,10.774169887619388,10.64447025011273,12.737064401588695,13.241162992699769,12.93346385267819,13.349562689736945,13.909061126012777,13.349562689736945,14.115460549153669,12.884163990464787,13.712661674923279,14.101060589399657,15.104057786155076,14.398659757649318,15.144257673801702,13.773661504436818,13.635061891804416,15.573656473688818,13.174963177719501,14.120960533781941,15.10625778000638,14.604659181908158,14.183460359103188,13.352962680234421,12.905063932052215,14.619459140544231,14.051360728304196,14.000560870283083,13.904261139428101,14.97395814976636,13.698761713771832,14.475059544122015,14.219860257370286,24.0085328993242,23.93003311872071,24.6156312025647,23.57363411480882,23.69143378557431,23.603334031801477,22.33823756757886,23.77463355304196,24.1149326019511,21.09934103013092,21.227140672947808,21.30714044935901,20.87784164919241,21.118840975631148,21.355740313528813,21.74873921514883,20.90244158043885,21.830138987647228,22.02173845215205,21.086841065066668,20.605742409673812,20.671342226330996,21.724039284181874,20.691642169595337,20.578842484855546,20.73124205891888,17.151952062561282,17.262551753449763,17.71695048346538,17.210151899900428,17.79215027329191,16.33945433338503,17.917849921978007,14.5406593607792,15.628456320530491,17.05285233953191,15.201757513097252,14.78815866905135,14.65795903294212,14.593459213210592,16.228854642496547,16.228854642496547,16.228854642496547,17.220351871392857,16.228854642496547,16.228854642496547,16.228854642496547,14.645859066759927,15.290857264075225,15.914955519803101,15.290857264075225,15.01545803377967,15.537256575421722,14.28666007067364,14.89315837559105,18.412048540758192,17.88645000973661,17.29155167239882,17.708050508339632,18.73114764891837,16.814553005547044,16.344854318292786,18.369848658701287,17.51115105864757,18.72934765394912,18.8842472210253,20.06574391889821,18.27614892057967,17.489851118178088,16.916952719353382,18.42164851392754,19.516045455232753,18.31504881185961,19.24034622577566,20.758141983737147,18.816447410516812,19.48074555389131,18.16434923304502,20.646742295084554,18.813347419180875,19.41644573360081,19.651045077926653,18.98774693175729,19.45874561537823,20.42294292057422,18.5263482213057,19.00204689179079,20.9042415754081,22.18023800916674,21.34084035517223,24.23913225482949,24.7685307752306,21.0806410823948,20.849941727169,21.914638751481558,21.1013410245412,23.56283414499331,21.62573955891661,21.829838988485687,21.46444000972753,22.109838205924888,23.035335619281955,23.172535235827162,22.609836808494887,22.69583656813693,20.04794396864672,20.413042948243334,21.95173864779225,21.319940413584803,22.254337802067617,22.768836364112147,21.517039862717894,21.96973859748477,22.535537016152983,21.812339037395738,21.564839729123587,22.340637560871198,25.1342297531503,26.21872672212463,27.28662373749364,30.17111567571997,28.53232025593654,27.819122249230688,28.17082126627843,29.42181776990857,29.95891626878926,26.42262615225268,26.821025038780455,25.99832733811178,23.756133604746868,24.39273182553899,26.17342684873179,23.496734329733552,27.524523072596445,26.67972543369417,25.023030063938734,25.20482955583319,28.00362173357902,32.50500915279621,35.00360216955902,12.184665945469359,10.064671870572559,12.18216595245651,12.086066221042554,11.848266885660262,11.60096757682914,11.618467527919089,12.330565537699284,11.319268364141202,10.16727158381992,10.169371577950717,11.675067369730014,10.159371605899317,9.989272081305002,12.217865852680006,10.992769276662992,13.672361787556136,12.477865126016406,16.106654984028438,12.48896509499346,15.837555736125266,13.145563259888386,16.64915346781689,13.783561476767705,12.04546633451387,13.62626191639918,16.615553561724184,13.59376200723213,13.67166178951254,13.151863242280767,16.584353648923816,12.13346608856619,11.496367869171495,13.074563458323446,10.826869740330265,11.9399666293716,11.782367069841536,11.262868521771306,10.991269280855281,10.48217070371851,8.902775117920392,9.60467315620816,17.78975027999957,10.657370214059036,9.916072285888756,10.83396972048676,9.511673416130138,10.04297193122102,10.200571490751084,9.76247271517925,10.60097037168914,15.947555428690665,17.51475104858607,18.908747152551232,18.733847641372247,19.011246866078082,19.535245401571444,18.217449084637952,18.841847339527366,20.798341871383776,18.08114946557737,19.76514475903313,18.223449067868792,18.040749578489713,18.62594794293764,19.5456453725049,18.93964706619006,19.41234574505974,19.21114630738557,18.32114879481097,18.21644908743281,18.078349473402977,16.076255068992182,15.407956936797119,14.995958088279442,14.391059778890254,14.416059709018754,14.1542604407131,15.273557312426304,15.70395610951856,15.096557807116525,14.902758348760392,13.75846154691869,14.922358293981135,15.10045779621657,14.699358917234916,14.173060388169734,14.415859709577726,13.983760917236733,14.106260574866381,13.902261145017821,20.053243953833963,22.72113649742697,18.49974829564897,22.041438397093312,17.90114996865217,18.81874740408863,20.977741369985893,20.794041883401675,19.389845807944088,21.709339325266317,17.73865042281692,17.45685121040847,17.72795045272192,19.31204602538419,20.38194303516348,18.375948641652638,21.012041274122193,22.063338335885877,17.923849905208847,18.299848854341487,15.9349554639059,16.74705319420009,16.352054298169797,17.29095167407574,16.19645473305001,15.603156391240448,17.32315158408125,15.70165611594674,15.70165611594674,17.72795045272192,17.412651333941277,16.735953225223042,15.19965751896646,15.915555518126183,16.59545361790087,16.91315272997385,17.4849511318729,17.77235032863014,16.85305289794493,16.740553212366684,17.77235032863014,17.688850562000948,16.16215482891371,16.761953152556682,19.67524501029104,15.940455448534168,16.82655297200872,16.04095516765074,16.773853119297847,16.385754203983012,20.20664352510244,18.649847876140488,17.885350012810957,19.425545708167583,16.24015461091463,21.66443945075553,21.03784120201481,20.13794371710932,19.65344507121899,21.480139965848227,21.812039038234193,22.05413836159859,20.57604249268115,21.23264065757608,20.836041766017555,21.0313412201814,21.24974060978397,24.867330499098436,24.9406302942352,24.867330499098436,24.87113048847797,23.438134493512347,22.997535724927662,25.73332807874968,23.990932948513738,23.06713553040541,23.56623413549078,22.561836942648167,23.85933331631732,25.3749290804275,22.23723784985972,22.803336267689478,21.9656386089437,25.44512888422833,28.47432041803842,24.25573220843481,22.9906357442122,23.205535143596784,27.53172305247345,26.791125122346767,26.23862666650692,32.25160986101374,29.54451742697924,29.264318210099017,37.47639525842921,40.67808631012595,37.089996338363115,8.327776724964892,8.532576152577565,8.833275312163162,9.024274778344902,9.01347480852939,8.717275636366923,8.71117565341557,8.98697488259318,8.645375837317355,10.476370719928696,10.793169834517048,9.058574682481204,8.59697597258858,8.508076221051633,11.390368165426656,8.987574880916265,9.27797406928892,8.50947621713883,10.435670833679499,9.160874396567024,8.68547572524347,11.066769069843351,10.55767049270658,12.826864150610264,11.258568533789203,13.544662144459759,10.019371997179714,10.733070002488134,12.649264646977402,12.340865508912225,13.51516222690813,14.282360082691538,10.141071657045254,12.42546527246707,11.20296868918342,12.74146438929131,12.322265560896621,10.236771389577152,12.05596630516784,13.174563178837445,9.588573201205405,10.714170055310989,13.805661415001298,10.966069351285755,9.821172551120968,10.555870497737326,11.20346868778599,12.459365177721315,13.217063060055896,12.095566194491385,13.585762029591013,11.978066522887435,13.032363576266535,15.121257738083482,15.04295795692102,14.989958105048599,16.86975285127077,16.072355079892134,15.666856213207865,16.358254280841663,15.388156992135348,24.248732227998833,15.086357835624096,17.01235245272374,17.56465090912256,17.305651632991296,16.675853393194128,15.963055385370335,15.130757711532313,16.78915307653649,14.918558304601603,15.273157313544248,15.16765760840198,19.77914471990509,15.095857809072925,20.52764262795238,20.09444383868573,19.015546854060183,19.621845159536566,14.908758331991232,20.397442991843153,19.34844592365129,20.543342584073077,14.309560006671346,14.645259068436841,19.797244669318122,19.13364652398722,21.522839846507708,21.81913901839069,20.32894318329106,19.93144429424791,19.93144429424791,21.278340529850976,16.09015503014363,17.076352273852695,18.66294783952782,16.73275323416659,18.4756483630051,18.08174946390045,18.48124834735388,19.65024508016254,20.400542983179083,18.468548382848603,16.59545361790087,15.748355985426777,15.494856693923785,16.981352539364398,16.098555006666803,17.730150446573226,18.868347265463576,17.45145122550071,17.389251399341003,17.12195214640708,17.859750084359373,17.25345177888299,19.16844642672609,17.25345177888299,18.229749050261173,18.70814771320015,19.2206462808344,17.38865140101792,19.176846403249268,17.73575043092201,17.803050242827933,17.390751395148712,19.51844544852509,17.65975064333137,10.734569998295845,11.072869052794704,9.82197254888508,11.660367410814455,9.925672259058098,11.486667896281638,10.786269853801581,9.907372310204035,10.89026956313614,11.787367055867236,10.692870114841506,11.486667896281638,11.159868809641885,11.61346754189339,11.64396745665016,11.099668977892458,9.834672513390359,8.824775335919472,13.025763594712611,12.743064384819533,13.024363598625415,18.290548880333684,12.625664712936098,11.6906673261302,13.025563595271583,18.41574853041721,13.04166355027434,19.467145591901406,12.8899639742546,19.11294658184082,19.076546683573724,19.33464596222036,18.9142471371795,20.199943543828,18.751547591903222,14.529059393199574,18.999546898777943,19.912144348188708,21.93093870592534,21.734839253997386,23.02413565058439,22.447537262100663,24.079032702286575,22.63143674812591,21.397640196424177,21.03304121543013,21.916138747289267,21.36644028362381,20.63534232694596,19.84144454578531,21.163840849862446,21.685039393181412,20.774841937062988,20.990141335329632,21.69433936718922,23.77683354689327,25.380229065614742,24.97733019166384,22.10743821263255,24.342731965281992,23.530934234149342,22.48193716595748,25.02983004493369,24.57913130457709,24.889530437052542,26.06652714750232,27.52712306532981,27.97532181267356,25.328529210109004,25.46642882469781,26.37002629926231,27.805722286681814,26.769625182436258,25.49992873107,25.52442866259593,27.422923356554218,25.4142289705895,24.12243258098965,27.970321826647854,26.195526786965385,28.93721912429772,26.25392662374556,26.61642561060881,28.64681993592507,25.331529201724425,24.41093177467254,26.57112573721597,27.07672432413475,24.89873041133983,26.291226519497283,28.564020167339475,24.25443221206813,24.40433179311862,25.71442813157253,28.17412125705539,24.552331379479337,34.33400404099727,34.29090416145574,33.7985055376448,31.93191075453048,32.546009038206954,22.83613617601807,32.25430985346762,22.574036908550873,27.273223774944764,26.970224621787345,27.26312380317285,27.490723167062708,27.04582441049593,30.442814916356507,30.31011528723443,29.19711839791361,28.31192087192368,28.6349199691839,27.35472354716367,26.6649254750581,28.397020634081095,27.161524087130623,30.41751498706646,28.712719751743794,27.24682384872907,29.026218875555184,27.43872331239543,28.775319576785556,27.55112299825317,28.248221049956264,27.79142232664831,28.31472086409807,28.89091925369974,28.89091925369974,28.89091925369974,28.89091925369974,28.012921707586823,28.759719620385376,29.51891749852766,28.023821677122847,29.48361759718622,29.604717258728673,29.775316781925557,30.237615489861778,29.96391625481496,29.67101707342945,29.245218263480844,29.740516879186682,29.13981855805909,28.81791945772452,29.570917353194943,29.18531843089296,29.239218280250004,29.750816850399627,29.131318581815396,30.214415554702533,28.92501915839501,30.28181536632897,29.480717605291314,29.79881671624635,34.45390370589356,33.41870659913264,33.23370711618173,33.09550750243138,34.42390378973936,33.455706495722815,31.62611160919867,34.47410364943738,33.037107665651206,34.343504014446104,32.354009574820076,35.32800126290643,35.95119952114969,34.31720408795093,36.12179904434657,35.30980131377289,32.36760953680998,33.494606387002754,32.441609329990335,32.441609329990335,33.14280737023451,36.19919882802441,37.11249627547876,35.69390024026716,35.819799888394286,36.43509816871693,36.45369811673253,36.153798954911046,36.77089723020294,34.60080329532863,35.12480183082199,32.53650906475812,33.10020748929554,33.14830735486278,33.12730741355484,33.07570755776961,37.22429596301342,36.07069918716392,36.900396868268565,36.0019993791708,37.50929516647832,36.33149845826442,36.64479758263479,36.47649805300972,38.15279336798591,37.11249627547876,41.05648525255092,41.81158314215213,42.983579866576214,40.504186796152105,40.47728687133384,41.32738449542335,42.21458201582355,40.801585964960736,40.20738762566655,41.16948493673174,40.57648659408372,39.62428925534942,8.384876565378386,6.715081232235614,8.83897529623246,7.731078392657854,6.764181095007989,7.050080295957515,6.896380725527496,7.3876793524127775,6.987280471474722,6.623181489083248,7.580778812725313,6.580281608982742,7.935977819991041,8.106077344585355,8.68097573782034,8.133577267726704,7.371079398807454,7.796978208476579,7.008880411105746,7.839278090254003,9.442473609534451,8.570876045534426,10.973769329765332,10.04447192702873,8.77347547929579,9.40197372272628,10.55697049466298,10.68347014111319,9.669072976219175,10.845569688066385,8.373876596121846,10.432370842902536,9.27247408466065,10.895069549720814,8.426076450230154,10.618570322499604,10.381870984042965,9.125674494946098,11.174268769395901,9.44917359080889,11.437168034627208,11.437168034627208,10.038371944077376,12.027766383982891,10.647370242007636,10.907669514505578,9.800272609533542,10.09547178449087,10.487370689185235,11.32396835100536,10.7149700530751,11.912166707068709,11.75346715061299,11.437168034627208,12.257865740885606,11.437168034627208,11.528367779735976,12.83466412881036,11.473567932894303,10.831369727753396,10.92446946755193,11.652067434011794,11.726267226633182,11.76276712462079,10.547170522052609,10.423170868615248,11.865066838706616,11.46296796251982,14.997558083807663,17.26175175568565,16.67475339626847,13.392262570396422,17.726950455516782,14.15016045217203,17.32385158212485,16.42445409582193,14.051560727745226,13.485062311033413,13.219363053627715,15.072857873354707,13.347162696444608,14.490159501919628,17.36885145635615,12.769364311314716,14.209260286995802,13.410362519809457,12.769364311314716,13.36346265088839,14.851758491298252,17.890849997439226,14.754258763797102,15.279157296775088,13.899261153402401,20.213643505538418,25.002330121792337,19.369745864120773,23.886833239458667,22.10613821626587,22.10613821626587,18.121149353782968,18.121149353782968,20.70314213745445,22.155038079597215,21.388240222695863,21.822339009447138,19.149846478710487,21.811539039631622,18.23684903041767,19.970144186086827,18.32324878894176,22.111438201453108,22.260137785857427,20.68084219977983,26.3085264711462,24.893530425873102,24.363331907707877,26.25542661955327,25.43122892307688,27.74822244738626,25.298429294234293,26.202126768519307,28.552420199759847,27.31842364861709,22.56073694572251,23.36723469166792,22.7099365287294,25.32462922100896,22.82443620871793,24.19173238730585,23.836133381158067,25.03943001810303,23.36723469166792,24.92323034286576,19.324645990168957,24.320332027886856,20.541342589662797,20.64324230486656,25.418028959969032,26.71052534761248,20.39014301224563,18.04974955333597,20.01744405388995,18.22014907709183,24.598731249797833,25.676028238895153,18.064249512810502,26.244926648899302,18.38124862683988,19.225246267978044,26.126326980369697,19.936144281112067,19.293046078486533,31.945710715961415,34.50370356670953,34.38730389203124,32.89430806475722,34.51790352702252,35.305301326349756,32.297709732170695,33.89460526905875,34.43080377045483,33.63580599236853,32.61890883446166,33.97660503988023,33.30950690433134,30.49831476124178,32.96380787051445,30.448014901823232,31.596011693323955,31.251812655314772,33.18090726375034,34.367403947648945,35.03840207229789,34.48160362847594,34.17700447979029,34.43470375955487,34.12050463769988,35.23790151472332,33.81160550103213,32.245909876944445,34.847002607234096,38.85839139593269,38.33169286798546,35.27220141885962,34.5627034018128,38.08859354741592,33.94840511869529,34.88510250074993,36.89889687246086,36.401598262344734,34.36300395994633,38.14669338503456,36.57379778106985,36.20879880119375,36.42589819442964,36.80309714020844,36.4291981852066,38.376692742216754,33.39610666229646,34.69030304518866,37.10389629951456,34.023404909080796,37.922294012201135,37.74469450856827,37.85979418687989,36.743997305384674,37.86429417430302,35.927999585990435,38.4705924797794,36.276598611702234,38.39049270364768,38.23779313042281,36.04789925088672,35.31600129644475,36.81859709688811,35.915499620926184,36.700697426402115,36.85929698313731,37.73079454741683,36.87899692807857,36.42939818464763,37.500095192191026,36.19529883892436,37.50949516591935,38.2741930286899,39.35779000017961,40.48808684114935,38.738791730197946,36.74889729168986,37.676494699177724,38.89409129615619,36.95869670532824,41.85788301275012,43.75567770866481,44.26247629222976,51.445856215632446,10.459670766602859,12.212865866654306,12.687064541331695,12.687064541331695,12.687064541331695,10.071571851288025,12.470565146418885,10.75866993093972,9.783872655369246,10.23917138286949,9.61247313440825,10.239571381751546,11.301568413610225,10.506270636362382,10.45897076855926,12.069166268275689,12.587064820817695,11.27266849438168,10.373871006401846,12.871464025959511,12.489865092478086,12.489865092478086,9.196174297908469,12.913363908854876,12.271765702037051,14.348059899069234,13.314662787277557,13.30016282780303,12.87296402176722,12.414365303490015,13.616661943229838,12.303065614557935,13.804861417237186,14.452759606447392,14.065360689176156,14.229860229421684,12.657864622941606,13.619661934845258,11.77396709331836,13.45776238733309,12.235165804328929,15.28945726798803,15.494856693923785,16.747753192243692,15.179357575702115,15.688456152838889,16.109354976482315,15.478856738641547,15.69545613327487,16.30195443819228,15.288657270223919,16.067855092469006,16.56985368944929,16.4449540385273,20.44344286327959,15.928755481234031,15.41295692282282,14.020760813826913,14.81365859778242,16.038555174358404,15.20295750974342,16.700853323322626,16.35875427944423,16.90535275177376,16.194854737521784,16.068355091071577,19.373445853779792,19.41214574561871,19.82044460447737,22.1299381497482,19.75264479396888,21.049941168197,23.86373330401993,21.71593930682024,19.78974469027957,19.43824567267286,18.90014717658703,17.98144974422491,21.08704106450769,21.419640134937257,19.522045438463593,19.899944382286,18.327348777482836,21.45694003068898,20.471742784185054,21.15744086774955,17.952049826393793,22.30513766008873,20.75094200386014,18.412048540758192,20.71724209804692,22.729036475347574,22.729036475347574,22.117838183566008,19.5206454423764,19.952444235555852,18.016349646684297,19.79604467267195,20.270343347069858,20.550342564509055,20.59234244712494,20.736342044665097,19.43434568357282,21.104341016156617,22.64333671486708,19.920544324711884,22.228737873616033,22.64333671486708,17.943349850709076,17.78835028391238,18.269648938746258,18.026549618176723,17.605550794812782]

In [47]:
len(distance)

2000

In [48]:

a = 100

while a < int(len(gpd_domain)/20):
    k = a*20
    for i, row in gpd_domain.iloc[k:k+20].iterrows():
        x = row['geometry'].x
        y = row['geometry'].y
        coor = [[x, y],[144.9628,-37.8102]]
        route = client.directions(coordinates=coor, profile='driving-car',format='geojson')
        dis = route['features'][0]['properties']['segments'][0]['distance']*0.000621371*1.60934
        distance.append(dis)
    time.sleep(30)
    a+=1

ApiError: 403 ({'error': 'Quota exceeded'})

In [127]:
distance 

[2.640392620451656,
 2.127094055053294,
 2.317093524029894,
 1.993894427328646,
 1.863594791498904,
 2.11449409026853,
 1.49949580910743,
 2.796792183335552,
 2.317093524029894,
 2.17249392816665,
 2.2368937481776663,
 2.050894268021626,
 2.278393632190976,
 3.287590811618264,
 1.72399518166136,
 2.201293847674682,
 2.2853936126269563,
 2.9067918759009523,
 2.186693888479638,
 1.72399518166136,
 4.195388274444356,
 4.797186592497607,
 4.44698757125758,
 4.211088230565054,
 3.604589925647644,
 5.425284837046042,
 5.905583494674785,
 5.537184524301208,
 5.341285071814282,
 5.365185005017128,
 5.059685858846858,
 5.898883513400346,
 5.800283788973542,
 5.99548324341687,
 5.597084356889094,
 5.147085614576095,
 5.8459836612484395,
 5.514184588582988,
 5.811283758230083,
 5.199185468963888,
 6.093582969241104,
 5.945883382041926,
 6.19548268444487,
 5.392684928158478,
 5.24148535074131,
 6.170782753477912,
 5.042685906359478,
 5.166685559796838,
 5.042685906359478,
 5.122085684447595,
 3.91

In [128]:
len(distance)

2000

In [129]:
import csv

In [130]:
with open('../data/distance_list', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(distance)